In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
#from itertools import produt
#from google.colab import files

In [2]:
DOMAIN = "https://dataserver-coids.inpe.br"
URL = f'{DOMAIN}/queimadas/queimadas/focos/csv/diario/Brasil/'
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

In [3]:
def get_page_content(url, headers):
  response = requests.get(url, headers=headers).text
  soup = BeautifulSoup(response, 'html.parser')
  return soup

In [33]:
def parse_csv_list(soup):
  table = soup.find('div', {'class': 'table'})
  list_csv = []
  rows = table.find_all('div', class_ = 'row')

  for row in rows:
    name_cell = row.find('div', class_ = 'name').find("a", href=lambda href: href and href.endswith(".csv"))
    date_cell = row.find_all('div', class_ = 'cell fixed')[-1].text


    if name_cell:
      file_name =  name_cell.get_text(strip=True)
      file_href = URL + name_cell['href']
      file_date = date_cell.format("%d/%m/%Y %H:%M:%S")

      # print(file_date)

      csv_data = {
        'name': file_name,
        'link': file_href,
        'date': file_date
      }
      list_csv.append(csv_data)

  return list_csv


In [34]:
soup_csv_list = get_page_content(URL, HEADERS)
list_csvs = parse_csv_list(soup_csv_list)

# URL da página contendo o link para o CSV
url = list_csvs[0]["link"]

# Fazer a requisição da página
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")


# Procurar pelo link do arquivo CSV
# Aqui, estou assumindo que o link possui o texto 'Download CSV' e termina em '.csv'
csv_link = soup.find("a", href=lambda href: href and href.endswith(".csv"))["href"]

# Verificar se o link é relativo e, se for, combiná-lo com a URL base
if not csv_link.startswith("http"):
    csv_link = url + csv_link

# Fazer o download do arquivo CSV
csv_response = requests.get(csv_link)
csv_content = csv_response.content

# Ler o CSV com o pandas (ou salvar o arquivo localmente)
df = pd.read_csv(pd.compat.StringIO(csv_content.decode('utf-8')))

# Mostrar as primeiras linhas do CSV
print(df.head())

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# if response.status_code== 200:
#     soup = BeautifulSoup(response.text, 'html.parser')
#     csv_link = soup.find('a', {'href': True})['href']
#     full_csv_link = f"https://dataserver-coids.inpe.br/{csv_link}"  # Adiciona o link completo ao csv_link
#     csv_data = requests.get(full_csv_link).content
#     df = pd.read_csv(pd.io.common.StringIO(csv_data.decode('utf-8')))
#     print(df.head())
# else:
#     print(f"Erro ao acessar a página: {response.status_code}")